In [0]:

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "2a9856d2-d354-4df2-be81-e74843ff3cc8",
"fs.azure.account.oauth2.client.secret": '-fq8Q~sL0_TQkSZkxECdesNl0QlEy-E1rh-ZSaYu',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d2a65ab9-e311-4bb8-830c-98dbb665e8b0/oauth2/token"}

dbutils.fs.mount(
source = "abfss://tokyo-olimpic-data@tokyoolimpicdatagerman.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolymic",
extra_configs = configs)
  



---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-1852161085365082>:7
      1 configs = {"fs.azure.account.auth.type": "OAuth",
      2 "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
      3 "fs.azure.account.oauth2.client.id": "2a9856d2-d354-4df2-be81-e74843ff3cc8",
      4 "fs.azure.account.oauth2.client.secret": '-fq8Q~sL0_TQkSZkxECdesNl0QlEy-E1rh-ZSaYu',
      5 "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d2a65ab9-e311-4bb8-830c-98dbb665e8b0/oauth2/token"}
----> 7 dbutils.fs.mount(
      8 source = "abfss://tokyo-olimpic-data@tokyoolimpicdatagerman.dfs.core.windows.net", # contrainer@storageacc
      9 mount_point = "/mnt/tokyoolymic",
     10 extra_configs = configs)

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<local

In [0]:
%fs
ls "/mnt/tokyoolymic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolymic/raw-data/,raw-data/,0,1692844956000
dbfs:/mnt/tokyoolymic/transformed-data/,transformed-data/,0,1692844968000


In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
athletes = spark.read.format("csv").option("header","true").load("/mnt/tokyoolymic/raw-data/athletes.csv")    
coaches = spark.read.format("csv").option("header","true").load("/mnt/tokyoolymic/raw-data/coaches.csv")   
gender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/gender.csv")   
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/medals.csv")   
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/teams.csv")   

In [0]:
athletes.show()

In [0]:
teams.show()

In [0]:
athletes.printSchema()

In [0]:
coaches.printSchema()

In [0]:
gender.show()

In [0]:
gender.printSchema()

In [0]:
gender = gender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))


In [0]:
gender.printSchema()

In [0]:
medals.show()

In [0]:
medals.printSchema()

In [0]:

# Find the top countries with the highest number of gold medals

top_gold_medal_countries = medals.orderBy("Gold",ascending = False).select("Team_Country", "Gold").show()


In [0]:

# Calculate the distribution of entries by gender for each discipline

average_by_gender = gender.withColumn(
    'dist_Female', gender['Female'] / gender['Total']
).withColumn(
    'dist_Male', gender['Male'] / gender['Total']
)
average_by_gender.show()



In [0]:
athletes.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/athletes")
coaches.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/coaches")
gender.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/gender")
medals.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/medals")
teams.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/teams")